In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import sys
import warnings
from training import train
from params import get_finetune_params
from task_configs.utils import load_task_config
from finetune_utils import seed_torch, get_exp_code, get_splits, get_loader, save_obj
from datasets.slide_datatset import SlideDataset
sys.path.append(os.path.abspath('../dino_stage2'))
from pprint import pprint

from dino_stage2.wsi_dataset import WSIDataset
from dino_stage2.eval_knn import extract_features
warnings.filterwarnings("ignore")

In [ ]:
tile_embedding_dir='/home/yuhaowang/WSI_Traing/data'
tile_embedding_dataset=WSIDataset(tile_embedding_dir,stage='test')


In [ ]:
next(iter(tile_embedding_dataset)).shape

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
test_case='/home/yuhaowang/WSI_Traing/data/TCGA-05-4422-01Z-00-DX1.4802093f-71ea-43d5-bd92-b4bd7fc8c5bf.npy'
ori_data=np.load(test_case,allow_pickle=True).item()
ori_data

In [ ]:
import os

len(os.listdir('/home/yuhaowang/data/test/TCGA-LUAD'))

## 将我们的数据处理为匹配provgigapath的，而不是修改gigapath的Dataset类

In [ ]:
import h5py
import torch
def shuffle_data(images: torch.Tensor, coords: torch.Tensor) -> tuple:
    '''Shuffle the serialized images and coordinates'''
    indices = torch.randperm(len(images))
    images_ = images[indices]
    coords_ = coords[indices]
    return images_, coords_

def read_assets_from_h5( h5_path: str) -> tuple:
    '''Read the assets from the h5 file'''
    assets = {}
    attrs = {}
    with h5py.File(h5_path, 'r') as f:
        for key in f.keys():
            assets[key] = f[key][:]
            if f[key].attrs is not None:
                attrs[key] = dict(f[key].attrs)
    return assets, attrs

def get_images_from_path(img_path: str,shuffle_tiles=False,max_tiles=1000000) -> dict:
    '''Get the images from the path'''
    if '.pt' in img_path:
        images = torch.load(img_path)
        coords = 0
    elif '.h5' in img_path:
        assets, _ = read_assets_from_h5(img_path)
        images = torch.from_numpy(assets['features'])
        coords = torch.from_numpy(assets['coords'])

        # if shuffle the data
        if shuffle_tiles:
            images, coords = shuffle_data(images, coords)

        if images.size(0) > max_tiles:
            images = images[:max_tiles, :]
        if coords.size(0) > max_tiles:
            coords = coords[:max_tiles, :]
    
    # set the input dict
    data_dict = {'imgs': images,
            'img_lens': images.size(0),
            'pad_mask': 0,
            'coords': coords}
    return data_dict

import os
test_path='/home/yuhaowang/data/embedding/TCGA-TOY'
file_list=os.listdir(test_path)

file_case=os.path.join(test_path,file_list[0])
data_dict=get_images_from_path(file_case)


{'imgs': tensor([[-0.0024,  0.0925, -0.1277,  ..., -0.0612, -0.0490,  0.0461],
        [ 0.0216,  0.0510, -0.1759,  ..., -0.0195, -0.0025,  0.1390],
        [ 0.0592,  0.1081, -0.1108,  ..., -0.0039,  0.0420,  0.1062],
        ...,
        [-0.0092, -0.0077,  0.0332,  ...,  0.0004, -0.0202, -0.0437],
        [-0.0183,  0.0641, -0.1471,  ..., -0.0219, -0.0289,  0.0578],
        [ 0.0275,  0.1116, -0.1241,  ..., -0.0008,  0.0439,  0.0720]]), 'img_lens': 7767, 'pad_mask': 0, 'coords': tensor([[ -168,   836],
        [ -168, 90950],
        [ -168, 91974],
        ...,
        [99162, 78662],
        [99162, 90950],
        [99162, 91974]])}


In [ ]:
data_dict

{'imgs': tensor([[-0.0024,  0.0925, -0.1277,  ..., -0.0612, -0.0490,  0.0461],
         [ 0.0216,  0.0510, -0.1759,  ..., -0.0195, -0.0025,  0.1390],
         [ 0.0592,  0.1081, -0.1108,  ..., -0.0039,  0.0420,  0.1062],
         ...,
         [-0.0092, -0.0077,  0.0332,  ...,  0.0004, -0.0202, -0.0437],
         [-0.0183,  0.0641, -0.1471,  ..., -0.0219, -0.0289,  0.0578],
         [ 0.0275,  0.1116, -0.1241,  ..., -0.0008,  0.0439,  0.0720]]),
 'img_lens': 7767,
 'pad_mask': 0,
 'coords': tensor([[ -168,   836],
         [ -168, 90950],
         [ -168, 91974],
         ...,
         [99162, 78662],
         [99162, 90950],
         [99162, 91974]])}

: 

In [ ]:
assets, _ = read_assets_from_h5('/home/yuhaowang/data/embedding/TCGA-TOY')
assets['coords']

array([[ 2016.,  4520.],
       [ 2272.,  4520.],
       [ 1760.,  4776.],
       [ 2016.,  4776.],
       [ 2272.,  4776.],
       [ 1760.,  5032.],
       [ 2016.,  5032.],
       [ 2272.,  5032.],
       [ 1760.,  5288.],
       [ 2016.,  5288.],
       [ 2272.,  5288.],
       [ 1760.,  5544.],
       [ 2016.,  5544.],
       [ 2272.,  5544.],
       [ 1760.,  5800.],
       [ 2016.,  5800.],
       [ 2272.,  5800.],
       [ 1760.,  6056.],
       [ 2016.,  6056.],
       [ 2272.,  6056.],
       [ 2528.,  6056.],
       [ 1760.,  6312.],
       [ 2016.,  6312.],
       [ 2272.,  6312.],
       [ 2528.,  6312.],
       [ 1504.,  6568.],
       [ 1760.,  6568.],
       [ 2016.,  6568.],
       [ 2272.,  6568.],
       [ 2528.,  6568.],
       [ 2784.,  6568.],
       [ 1504.,  6824.],
       [ 1760.,  6824.],
       [ 2016.,  6824.],
       [ 2272.,  6824.],
       [ 2528.,  6824.],
       [ 2784.,  6824.],
       [ 1504.,  7080.],
       [ 1760.,  7080.],
       [ 2016.,  7080.],


In [6]:
import os
len(os.listdir('/home/yuhaowang/data/embedding/TCGA-LUAD'))

538

In [ ]:
import os
len(os.listdir('/home/yuhaowang/data/processed_data/TCGA-LUAD/output'))

541

: 

In [13]:
file_list
assets, _ = read_assets_from_h5(os.path.join(test_path,file_list[0]))
assets['coords']

array([[b'07784', b'28596'],
       [b'07784', b'29620'],
       [b'07784', b'30644'],
       ...,
       [b'99945', b'70580'],
       [b'99945', b'71604'],
       [b'99945', b'72628']], dtype=object)

In [18]:
assets['coords'][0][0]

b'07784'

In [ ]:
img=torch.rand(678,224)
img_len=[678]
pad_mask=[0]
coords=torch.rand((678,2))
data_dict={'features':img,'img_lens':img_len,'pad_mask':pad_mask,'coords':coords}
#save h5 file
with h5py.File('/home/yuhaowang/data/temp/GigaPath_PANDA_embeddings/h5_files/test.h5', 'w') as f:
    for key,value in data_dict.items():
        f.create_dataset(key, data=value)

In [ ]:
data,_=read_assets_from_h5('/home/yuhaowang/data/test/TCGA-LUAD-AllFeature/TCGA-75-5147-01Z-00-DX1.3156FA5C-9737-4EAA-989C-FD578FBF15A1.h5')
data.keys()

In [ ]:

from collections import defaultdict, deque
def element_indices(lst):
    index_dict = defaultdict(list)  
    for i, value in enumerate(lst):
        index_dict[value].append(i)  
    return dict(index_dict)

In [ ]:
data['features'].shape

In [ ]:
data['features'].sum()

In [ ]:
data['features'][0].sum()
for i in data['features']:
    print(i.sum())


In [ ]:
len(os.listdir('/home/yuhaowang/data/test/TCGA-LUAD-AllFeature'))

In [ ]:
# align the name of feature dir

import  os
data_dir='/home/yuhaowang/data/test/TCGA-LUAD-AllFeature'
for file in os.listdir(data_dir):
    if file.endswith('.h5'):
        file_path = os.path.join(data_dir, file)
        #rename the file
        #TCGA-97-8552-01A-01-TS1.24ca90a7-5762-4ab5-a404-67dfcd416f79.h5 -> TCGA-97-8552-01A-01-TS1.h5
        new_file_name = file.split('.')[0] + '.h5'
        new_file_path = os.path.join(data_dir, new_file_name)
        os.rename(file_path, new_file_path)
        print(f'Renamed file: {file_path} to {new_file_path}')

In [11]:
import os
import sys
import argparse

import torch
from torch import nn
import torch.distributed as dist
import torch.backends.cudnn as cudnn
from torchvision import datasets
from torchvision import transforms as pth_transforms
from torchvision import models as torchvision_models 
import pandas as pd
import numpy as np
import json
import os
from PIL import ImageFile
import collections
from collections import Counter
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
import json
import torch
sys.path.append('/home/yuhaowang/project/FMBC/dino_stage2')
import vision_transformer as vits
embed_dim=384
checkpoint_path='/home/yuhaowang/project/FMBC/dino_stage2/output/checkpoint0180.pth'
model = vits.__dict__['vit_small'](embed_dim=embed_dim)
len(list(model.named_parameters()))

147

In [8]:
len(model)

NameError: name 'model' is not defined

In [ ]:
def load_pretrained_weights(model, pretrained_weights, checkpoint_key):
    if os.path.isfile(pretrained_weights):
        state_dict = torch.load(pretrained_weights, map_location="cpu")
        if checkpoint_key is not None and checkpoint_key in state_dict:
            print(f"Take key {checkpoint_key} in provided checkpoint dict")
            state_dict = state_dict[checkpoint_key]
        # remove `module.` prefix
        state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
        # remove `backbone.` prefix induced by multicrop wrapper
        state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
        msg = model.load_state_dict(state_dict, strict=False)
        print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))
        model.eval()
load_pretrained_weights(model, checkpoint_path, 'teacher')


Take key teacher in provided checkpoint dict
Pretrained weights found at /home/yuhaowang/project/FMBC/dino_stage2/output/checkpoint0180.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


VisionTransformer(
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=384, out_features=384, bias=True)
        )
      )
      (drop_path): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)

In [6]:
len(model)

NameError: name 'model' is not defined

In [3]:
test_tensor=torch.rand((1,17770,384))
mask=torch.ones((1,17770))

with torch.no_grad():
    output=model((test_tensor.cuda(),mask.cuda()))
print(output.shape)

/home/yuhaowang/anaconda3/envs/gigapath/lib/python3.9/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402412426/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


RuntimeError: Mask shape should match input. mask: [1, 17770] input: [1, 6, 17771, 17771]

In [5]:
a=5
def change(a):
    a=6
change(a)
print(a)

5


In [4]:
from pathlib import Path
import sys
str(Path(__file__).resolve().parents[1]

SyntaxError: unexpected EOF while parsing (1007196212.py, line 3)

In [ ]:
 = os.path.join(args.split_dir,args.task_code) if not args.pre_split_dir else args.pre_split_dir

In [ ]:
temp =3 if not 1<4 else

In [12]:
import pandas as pd
data=pd.read_csv('/home/yuhaowang/project/FMBC/finetune/dataset_csv/mutation/BRCA-6-gene_TCGA.csv')
#delete rows with nan
data=data.dropna()

In [14]:
#  EGFR: 0
#   FAT1: 1
#   KRAS: 2
#   LRP1B: 3
#   TP53: 4
#   HRD: 5
df=data[['EGFR','FAT1','KRAS','LRP1B','TP53','HRD']]

In [17]:
df.describe()

,EGFR,FAT1,KRAS,LRP1B,TP53,HRD
count,932,932,932,932,932,932
unique,2,2,2,2,2,2
top,False,False,False,False,False,False
freq,923,904,926,906,602,481


In [11]:
data.drop(columns=['HRP'],inplace=True)
data.to_csv('/home/yuhaowang/project/FMBC/finetune/dataset_csv/mutation/BRCA-6-gene_TCGA.csv')

In [17]:
data['file_name'][0].split('.')[0]

'TCGA-AN-A0FL-01Z-00-DX1'

In [18]:
data['slide_id']=data['file_name'].apply(lambda x:x.split('.')[0])
data.to_csv('/home/yuhaowang/project/FMBC/finetune/dataset_csv/mutation/BRCA-6-gene_TCGA.csv')

In [9]:
#conver HDR_label column to new columns with true and false
df=pd.get_dummies(data['HDR_label'])
data=pd.concat([data,df],axis=1)


In [11]:
data.drop(columns=['HDR_label'],inplace=True)
data.to_csv('/home/yuhaowang/project/FMBC/finetune/dataset_csv/mutation/BRCA-5-gene_TCGA.csv',index=False)

In [2]:
data_dir='/home/yuhaowang/data/embedding/TCGA-BRCA'
import os
import shutil

for file in os.listdir(data_dir):
    if file.endswith('.svs.h5'):
        new_file_name=file.split('.')[0]+'.h5'
        shutil.move(os.path.join(data_dir,file),os.path.join(data_dir,new_file_name))
        

In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 0, 0, 0])
y_scores = np.array([1, 0, 0, 0])
try:
    roc_auc_score(y_true, y_scores) ## y_true=ground_truth
except ValueError:
    pass


In [4]:
roc_auc_score(y_true, y_scores)

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [2]:
import os
len(os.listdir('/home/yuhaowang/data/embedding/TCGA-LUAD'))

538

In [2]:
import torch
torch.ones(3)

tensor([1., 1., 1.])

In [3]:
mask=torch.ones((1,0))
mask = torch.cat([torch.zeros_like(mask[:, :1]), mask], dim=1)

In [4]:
mask.shape

torch.Size([1, 0])